In [3]:
# load module and pre generate data
from graph_tree_module import Node, sim, graph_tree

import numpy as np

import pickle

# Open the file in binary read mode and load the object
with open("graph_tree.pkl", "rb") as file:
    gt = pickle.load(file)


In [4]:
sliders_data = [{"id": gt.attr_name[i], "min": gt.X_maxmin[1][i], "max": gt.X_maxmin[0][i], "value": gt.root.center[i], "type": "f"} for i in range(len(gt.attr_name))]
# print(sliders_data)

recom_data = [{"i": i, "id": gt.attr_name[i], "max_val": gt.recom[i][0].center[i], "min_val": gt.recom[i][1].center[i]} for i in range(len(gt.attr_name))]
# print(recom_data)

status = gt.get_suround_cluster(gt.root, gt.root.center)
print(status)
def get_suround_node_data(status):
    cur_node = status["cur_cluster"]
    cur_cluster = [{"id": cur_node.id, "position": "center", "x": 300, "y": 200, "color": "red"}]

    contra_node = status["contra_cluster"]
    contra_cluster = [{"id": contra_node[i].id, "position": "left", "x": 100, "y": 150+i*100, "color": "green"} for i in range(len(contra_node))]

    super_node = status["super_cluster"]
    super_cluster = [{"id": super_node.id, "position": "top", "x": 300, "y": 100, "color": "dark red"}] if super_node else []

    sim_node = status["neibour_cluster"]
    sim_cluster = [{"id": sim_node[i].id, "position": "right", "x": 500, "y": 200+i*100, "color": "blue"} for i in range(len(sim_node))] if sim_node else []

    sub_node = status["sub_cluster"]
    sub_cluster = [{"id": sub_node[i].id, "position": "bottom", "x": 200+i*100, "y": 350, "color": "pink"} for i in range(len(sub_node))] if sub_node else []

    return cur_cluster + contra_cluster + super_cluster + sim_cluster + sub_cluster

graph_data = get_suround_node_data(status)

print(graph_data)

[0.43906465 0.52589225 0.66144496 0.47741818 0.7539839  0.639
 0.06208811 0.25001819 0.13598254 0.1839972  0.47853415 0.48415781
 0.14202648 0.7291    ] [0.43906465 0.52589225 0.66144496 0.47741818 0.7539839  0.639
 0.06208811 0.25001819 0.13598254 0.1839972  0.47853415 0.48415781
 0.14202648 0.7291    ]
[0.43906465 0.52589225 0.66144496 0.47741818 0.7539839  0.639
 0.06208811 0.25001819 0.13598254 0.1839972  0.47853415 0.48415781
 0.14202648 0.7291    ] [0.43906465 0.52589225 0.66144496 0.47741818 0.7539839  0.639
 0.06208811 0.25001819 0.13598254 0.1839972  0.47853415 0.48415781
 0.14202648 0.7291    ]
[0.43906465 0.52589225 0.66144496 0.47741818 0.7539839  0.639
 0.06208811 0.25001819 0.13598254 0.1839972  0.47853415 0.48415781
 0.14202648 0.7291    ] [0.43906465 0.52589225 0.66144496 0.47741818 0.7539839  0.639
 0.06208811 0.25001819 0.13598254 0.1839972  0.47853415 0.48415781
 0.14202648 0.7291    ]
[0.43906465 0.52589225 0.66144496 0.47741818 0.7539839  0.639
 0.06208811 0.250018

In [5]:
from flask import Flask, render_template, request, jsonify

app = Flask(__name__)

@app.route('/')
def index():
    return render_template('index.html')

# slider ####################################################################
@app.route('/sliders')
def sliders():
    return jsonify(sliders_data)

@app.route('/handle_slider', methods=['POST'])
def submit_slider_values():
    data = request.json
    print('Received slider values:', data)
    query = np.array([float(val) for val in data.values()])
    cur = gt.search(query)
    print(cur.id)
    print(gt.status)
    global graph_data
    graph_data = get_suround_node_data(gt.status)
    # Process the data as needed
    return jsonify({'status': 'slider respon success', 'received': data})
# slider end ################################################################

# recom ####################################################################
@app.route('/recom')
def recom():
    return jsonify(recom_data)

@app.route('/handle_recom', methods=['POST'])
def handle_recom():
    data = request.json
    button_id = int(data.get('buttonId'))
    val = int(data.get('val'))
    print(button_id, val)
    moved_node = gt.recom[button_id][val]
    gt.get_suround_cluster(moved_node)
    global graph_data
    graph_data = get_suround_node_data(gt.status)
    # Handle the button click based on the button_id
    return jsonify({'status': 'recom respon success', 'buttonId': button_id, 'val': val})
# recom end ####################################################################

@app.route('/graph-data')
def graph():
    return jsonify(graph_data)

@app.route('/tree-data')
def tree_data():
    tree = {
        "name": "Root",
        "children": [
            {
                "name": "Child 1",
                "children": [
                    {"name": "Grandchild 1"},
                    {"name": "Grandchild 2"}
                ]
            },
            {
                "name": "Child 2",
                "children": [
                    {"name": "Grandchild 3"},
                    {"name": "Grandchild 4"}
                ]
            }
        ]
    }
    return jsonify(tree)

if __name__ == '__main__':
    app.run(port=8123)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8123
Press CTRL+C to quit
127.0.0.1 - - [05/Sep/2024 03:43:06] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [05/Sep/2024 03:43:06] "GET /static/styles.css HTTP/1.1" 304 -
127.0.0.1 - - [05/Sep/2024 03:43:06] "GET /static/scripts.js HTTP/1.1" 200 -
127.0.0.1 - - [05/Sep/2024 03:43:06] "GET /sliders HTTP/1.1" 200 -
127.0.0.1 - - [05/Sep/2024 03:43:06] "GET /recom HTTP/1.1" 200 -
127.0.0.1 - - [05/Sep/2024 03:43:06] "GET /graph-data HTTP/1.1" 200 -
127.0.0.1 - - [05/Sep/2024 03:43:06] "GET /tree-data HTTP/1.1" 200 -
127.0.0.1 - - [05/Sep/2024 03:43:12] "GET /graph-data HTTP/1.1" 200 -
127.0.0.1 - - [05/Sep/2024 03:43:12] "POST /handle_slider HTTP/1.1" 200 -


Received slider values: {'Popularityslider': '0.44', 'danceabilityslider': '0.5299', 'energyslider': '0.66301', 'keyslider': '0.48', 'loudnessslider': '0.756', 'modeslider': '0.64', 'speechinessslider': '0.0625', 'acousticnessslider': '0.25', 'instrumentalnessslider': '0.14', 'livenessslider': '0.1836', 'valenceslider': '0.4759', 'temposlider': '34.132', 'duration_in min/msslider': '0.50165', 'time_signatureslider': '1'}
checking 0, dist: 2.5163251533130255
0 potential
[1, 2] 

checking 1, dist: 2.839820126639239
1 potential
[2, 3, 4] 

checking 2, dist: 3.2697791337131052
2 potential
[3, 4, 5, 6] 

checking 3, dist: 3.1801362701851095
3 potential
[4, 5, 6, 7, 8, 9] 

checking 4, dist: 3.646591336243201
4 potential
[5, 6, 7, 8, 9, 10, 11] 

checking 5, dist: 3.630950009765608
5 potential
[6, 7, 8, 9, 10, 11, 14, 12, 13] 

checking 6, dist: 4.050317894775805
6 potential
[7, 8, 9, 10, 11, 14, 12, 13, 16, 15] 

checking 7, dist: 3.538089249129034
7 potential
[8, 9, 10, 11, 14, 12, 13, 16,

127.0.0.1 - - [05/Sep/2024 03:43:17] "POST /handle_slider HTTP/1.1" 200 -
127.0.0.1 - - [05/Sep/2024 03:43:17] "GET /graph-data HTTP/1.1" 200 -


Received slider values: {'Popularityslider': '0.44', 'danceabilityslider': '0.5299', 'energyslider': '0.66301', 'keyslider': '0.48', 'loudnessslider': '0.756', 'modeslider': '0.64', 'speechinessslider': '0.5025', 'acousticnessslider': '0.71', 'instrumentalnessslider': '0.86', 'livenessslider': '0.1836', 'valenceslider': '0.4759', 'temposlider': '34.132', 'duration_in min/msslider': '0.50165', 'time_signatureslider': '1'}
checking 0, dist: 4.1455046121434265
0 potential
[1, 2] 

checking 1, dist: 4.461875680874848
1 potential
[2, 3, 4] 

checking 2, dist: 4.882856600052566
2 potential
[3, 4, 5, 6] 

checking 3, dist: 4.806472691403128
3 potential
[4, 5, 6, 7, 8, 9] 

checking 4, dist: 4.253382294567721
4 potential
[5, 6, 7, 8, 9, 10, 11] 

checking 5, dist: 5.226237804933075
5 potential
[6, 7, 8, 9, 10, 11, 14, 12, 13] 

checking 6, dist: 4.636831508946468
6 potential
[7, 8, 9, 10, 11, 14, 12, 13, 16, 15] 

checking 7, dist: 5.165836925368011
7 potential
[8, 9, 10, 11, 14, 12, 13, 16, 1

...

6 0
[0.63860077 0.74230837 0.66410239 0.87980476 0.77922055 0.
 0.14881569 0.16375259 0.00897897 0.13725806 0.65526966 0.4516187
 0.13431518 0.7466443 ] [0.48842944 0.70722638 0.70365915 0.868045   0.78315387 0.
 0.10980067 0.15543423 0.01980725 0.16204362 0.70021365 0.45958429
 0.13929375 0.74832776]
[0.63860077 0.74230837 0.66410239 0.87980476 0.77922055 0.
 0.14881569 0.16375259 0.00897897 0.13725806 0.65526966 0.4516187
 0.13431518 0.7466443 ] [0.47678936 0.58652752 0.74931653 0.85786307 0.79420531 0.
 0.08568628 0.10873536 0.02346942 0.18740648 0.54260227 0.48446861
 0.14968165 0.74150268]
[0.63860077 0.74230837 0.66410239 0.87980476 0.77922055 0.
 0.14881569 0.16375259 0.00897897 0.13725806 0.65526966 0.4516187
 0.13431518 0.7466443 ] [0.48842944 0.70722638 0.70365915 0.868045   0.78315387 0.
 0.10980067 0.15543423 0.01980725 0.16204362 0.70021365 0.45958429
 0.13929375 0.74832776]
[0.63860077 0.74230837 0.66410239 0.87980476 0.77922055 0.
 0.14881569 0.16375259 0.00897897 0.1372

127.0.0.1 - - [05/Sep/2024 03:44:33] "POST /handle_recom HTTP/1.1" 200 -
127.0.0.1 - - [05/Sep/2024 03:44:33] "GET /graph-data HTTP/1.1" 200 -


8 0
[0.36255113 0.41942888 0.29943712 0.52742299 0.54153911 0.
 0.03399952 0.74220684 0.8157589  0.13389376 0.21184547 0.42472946
 0.12098704 0.68595041] [0.37621543 0.48923805 0.39491356 0.50446049 0.64939757 0.
 0.04152185 0.68021795 0.2510296  0.15254855 0.34990626 0.43908319
 0.12710863 0.70560748]
[0.36255113 0.41942888 0.29943712 0.52742299 0.54153911 0.
 0.03399952 0.74220684 0.8157589  0.13389376 0.21184547 0.42472946
 0.12098704 0.68595041] [ 4.37091133e-01  5.40213790e-01  6.74770868e-01  5.43188114e-01
  7.59257990e-01 -2.55351296e-15  7.01391100e-02  2.33917251e-01
  1.38365244e-01  1.80350134e-01  4.82980264e-01  4.82786358e-01
  1.50391476e-01  7.31578947e-01]
[0.36255113 0.41942888 0.29943712 0.52742299 0.54153911 0.
 0.03399952 0.74220684 0.8157589  0.13389376 0.21184547 0.42472946
 0.12098704 0.68595041] [0.37621543 0.48923805 0.39491356 0.50446049 0.64939757 0.
 0.04152185 0.68021795 0.2510296  0.15254855 0.34990626 0.43908319
 0.12710863 0.70560748]
[0.36255113 0.419

127.0.0.1 - - [05/Sep/2024 03:44:36] "POST /handle_recom HTTP/1.1" 200 -
127.0.0.1 - - [05/Sep/2024 03:44:36] "GET /graph-data HTTP/1.1" 200 -


0 1
[0.27265133 0.65713275 0.69860278 0.68421053 0.7815268  1.
 0.07244355 0.10668186 0.01853785 0.14165524 0.75270519 0.46148395
 0.13974785 0.7406015 ] [0.49171717 0.62277607 0.72619186 0.70396694 0.77905357 1.
 0.06700413 0.14214463 0.02870229 0.19458671 0.71778087 0.4763506
 0.13137124 0.74272727]
[0.27265133 0.65713275 0.69860278 0.68421053 0.7815268  1.
 0.07244355 0.10668186 0.01853785 0.14165524 0.75270519 0.46148395
 0.13974785 0.7406015 ] [0.47134111 0.52212517 0.74900149 0.7087966  0.78768744 1.
 0.06280401 0.10119061 0.02865018 0.20309752 0.52117946 0.50408515
 0.14839712 0.73696043]
[0.27265133 0.65713275 0.69860278 0.68421053 0.7815268  1.
 0.07244355 0.10668186 0.01853785 0.14165524 0.75270519 0.46148395
 0.13974785 0.7406015 ] [0.49171717 0.62277607 0.72619186 0.70396694 0.77905357 1.
 0.06700413 0.14214463 0.02870229 0.19458671 0.71778087 0.4763506
 0.13137124 0.74272727]
[0.27265133 0.65713275 0.69860278 0.68421053 0.7815268  1.
 0.07244355 0.10668186 0.01853785 0.141